In [1]:
 def loadWheel(round):        #method for loading appropriate wheel (list) for each round
    import csv
    if round == 1:
        filePath = 'r1_wheel.csv'
        #filePath = 'test r1.csv'
    else:
        filePath = 'r2_wheel.csv'
        #filepath = 'test r2.csv'

    with open(f'{filePath}', mode='r') as f:
        sample_lines = f.read().splitlines()              
        return sample_lines

In [2]:
def loadList():             #method for loading word/hint list csv as lists
    import csv
    with open('phrase_hint_list.csv', mode='r') as f:
        csv_reader = csv.reader(f, delimiter=',')
        line_count = 0
        for row in csv_reader:
            wordList.append(row[0])
            hintList.append(row[1])

In [3]:
def spin(wheel):          #simulate spinning wheel. Returns 1-5 to signal strings. returns 100-1000 as potential cash prize
    import random
    result = random.choice(wheel)
    print(f'The wheel landed on: {result}')
    try:
        return int(result)
    except: 
        if result == 'lose_turn':
            return 1
        elif result == 'bankrupt':
            bankrupt = True
            print('br')
            return 2
        elif result == 'million':
            million = True
            print('million')
            return 3
        elif result == 'mystery':
            return 4
        elif result == 'jackpot':
            return 5


In [4]:
def getGuess(guessedLetters):       #validates that guesses were not previously guessed
    goodGuess = False
        
    while goodGuess == False:
        guessLetter = input('Enter Guess')
        
        if guessLetter in guessedLetters:             
            print("You have previously guessed that letter or phrase")
               
        else:  
            goodGuess=True         #new guess makes goodGuess True, breaking out of while loop
    return guessLetter             #returns the validated letter

In [5]:
def checkLetter(guess, selectedWord, mask):                 #function for checking presence of letter in phrase
    if len(guess) == 1 and selectedWord.find(guess)!=-1:    #if single letter that's in the phrase...
        for index, i in enumerate(selectedWord):
            if i == guess:                                  #update the 'masked' phrase to show user
                mask[index] = guess                 
        print(*mask)
        if '_' not in mask:                                 #if word is unmasked, the round has been solved, and the current player wins
            gameOver = True
        else:
            return True                                     #returns True if correct guess
    else:
        return False

In [6]:
def remainingConsonants(selectedWord, guessedLetters, vowels):      #counts remaining consonants
    count = 0
    for x in selectedWord:
        if x not in vowels and x not in guessedLetters:
            count+=1
    return count
            
        
    

In [7]:
def remainingVowels(selectedWord, guessedLetters, vowels):         #counts remaining vowels
    count = 0
    for x in selectedWord:
        if x in vowels and x not in guessedLetters:
            count+=1
    return count
            
        
    

In [8]:
def guessVowel(selectedWord, guessedLetters, vowels, player, mask):        #process for guessing a vowel
    global bank
    continueGuess = True
    while continueGuess == True and remainingVowels(selectedWord, guessedLetters, vowels) > 0:    #only allows guessing, if vowels are remaining
        vowelPrompt = input('Type "yes" to buy vowel for $250')                             
        if vowelPrompt == 'yes':                      
            if bank[player] > 250:                                         #user needs to explicity buy vowel. User can only buy with available funds
                bank[player] = bank[player]-250
                print(f'New Balance: ${bank[player]}')
                
                isVowel = False                                            #check that guess is actually a vowel
                while isVowel == False:
                    guess = getGuess(guessedLetters).lower()
                    if len(guess) ==1 and guess in vowels:
                        isVowel = True
                    else:
                        print('Please enter a vowel')      
                guessedLetters.append(guess) 
                        
                if checkLetter(guess, selectedWord, mask) == True:        #checks letter in phrase. Can continue with vowels if correct guess
                    continueGuess = True    
                else: 
                    continueGuess = False
            else:
                print('Insufficient funds')
        else:
            print('Vowels not purchased')                              #Ends 'purchase' vowel phase
            continueGuess = False

            
    

In [9]:
def runRound():                        #function for running a normal round
    import random
    global roundWinner
    
    #init random word, various variables
    mask = []                                         
    guessedLetters=[]
    randNum = random.randrange(0,len(wordList))
    selectedWord = wordList[randNum].lower()
    selectedHint = hintList[randNum].lower()
    for i in selectedWord:                          #setup masked phrase
        if i == ' ':
            mask.append(' ')
        else:
            mask.append('_')
    gameOver = False                  
    jackpotActive = False                        #jackpot related variables
    jackpotAmount = 5000
    
    print(*mask)
    print(f'Hint: {selectedHint}')
    
    for index, x in enumerate(turnList):       #sets up player turn list
        turnList[index] = 0
    
    while gameOver != True:
        for player, turn in enumerate(turnList):    #iterate through player turns
            #process turn
            if gameOver == True:
                break

            turnList[player] = turnList[player] + 1   #record player's turn

            if remainingConsonants(selectedWord, guessedLetters, vowels) > 0:   #if consonants remain: spin
                prompt = input(f'Player {player + 1}: Type anything to spin')
                result = spin(wheel)                                            # 1-5 signals string action, 100-900 are $ amounts

                if result == 1:                                                 #actions for string cases (bankrupt, jackpot, etc.)
                    print(f'Sorry, Player {player + 1} has lost their turn')
                elif result == 4:  
                    print('Mystery! 50/50 chance of $1000 or Bankrupt')
                    option = random.choice(['bankrupt', 'prize'])
                    if option == 'bankrupt':
                        bank[player] = 0
                        millions[player] = False
                        print(f'Sorry, you are bankrupt!')     
                    else:
                        bank[player] = bank[player] + 1000    
                        print(f'Congradulations, you won $1000!')  
                elif result == 2:
                    bank[player] = 0
                    millions[player] = False
                    print(f'Sorry, you are bankrupt!')
                elif result == 3:
                    millions[player] = True     
                    print(f'Become the champion to win a million dollars!')
                elif result == 5:
                    jackpotEligible[player] = True
                    jackpotTrack = True
                    
                    
                else:                                                         #start guessing for prize money
                    print(*mask)
                    print(f'Player {player+1}: For the chance of winning ${result}, guess a consonant, or the entire phrase')
                    
                    if jackpotActive == True:                                
                        jackpotAmount = jackpotAmount + result                #Adds to spins to jackpot, if active
                    
                    isConsonant = False
                    while isConsonant == False:                               #validate guess is consonant
                        guess = getGuess(guessedLetters).lower()   
                        if guess in vowels:
                            print('Please enter a consonant or word')
                        else:
                            isConsonant = True
                    guessedLetters.append(guess)    

                    if guess == selectedWord:                                  #check phrase guess
                        bank[player] = bank[player] + result
                        gameOver = True

                    elif checkLetter(guess, selectedWord, mask) == True:        #check letter guess       
                        bank[player] = bank[player] + result
                        print(f'Correct Guess! You have earned ${result}. New Balance: ${bank[player]}')      #pay out if correct guess
                        guessVowel(selectedWord, guessedLetters, vowels, player, mask)                            
                    else:            
                        print('Incorrect guess, you earned $0')

            elif remainingVowels(selectedWord, guessedLetters, vowels) > 0:      #guess vowels (if available)
                print(f'Player {player+1}: No consonants remaining. You have the option to buy a vowel for $250.')
                guessVowel(selectedWord, guessedLetters, vowels, player, mask)    


            if gameOver == False and result!=1:                                 #if word hasn't been solved, and turn not skipped...
                print(f'Player {player+1}: You have a chance to guess the phrase!')     #option to guess whole phrase
                guess = getGuess(guessedLetters).lower()
                if guess == selectedWord: 
                    gameOver = True                                             #if correct guess, word is solved
                    print('Correct guess!')
                else:
                    print('Incorrect guess')
                     
            
            if gameOver == True:                                              #if word is solved, current player is declared winner
                roundWinner = player
                bank[player] = bank[player] + 2500
                    
                if jackpotEligible[player] == True:                           #if player solves phrase, and has active jackpot, pay out jackpot
                    bank[player] = bank[player] + jackpotAmount
                    
            if turnList[len(turnList)-1] == numTurns:                         #if turns expire, ends game
                    gameOver = True 
                    
            print(f'Current earnings for all players: {bank}') 
            print('---------------------')

    if roundWinner is None:                         #if game ends by turn expiration, round winner is the individual with largest bank
        maxMoney = 0
        for index, money in enumerate(bank):
            if money > maxMoney:
                maxMoney = money
        roundWinner = bank.index(maxMoney)

    print(f'Round over! Player #{roundWinner+1} wins this round!')     #print winner of round and cash
    print(f'Player {roundWinner+1}: ${bank[roundWinner]}')
    print('----------------')
        

In [10]:
def runFinalRound():               #function for final round
    import random
    
    print(f'Player {roundWinner+1}: Please play in the final round')        #round winner only plays
    #init random word, various variables
    mask = []
    guessedLetters=['r', 's', 't', 'l', 'n', 'e']
    
    randNum = randNum = random.randrange(0,len(wordList))                   #select random phrase                                                      
    selectedWord = wordList[randNum].lower()
    selectedHint = hintList[randNum].lower()
    
    for i in selectedWord:                                                 #init masked phrase
        if i == ' ':
            mask.append(' ')
        else:
            mask.append('_')
    gameOver = False

    for guess in guessedLetters:                                #reveal free letters
        checkLetter(guess, selectedWord, mask)
    print(f'Hint: {selectedHint}')
    
    
    count = 1
    while count < 4:
        isConsonant = False
        while isConsonant == False:                                   #guess consonant, guess consonants one at a time
            print(f'Guess three consonants, one at a time. [r, s, t, l, n, e] are given. Guess {count}: ')
            guess = getGuess(guessedLetters).lower()
            if guess in vowels:
                print('Please enter a consonant or phrase')
            else:
                isConsonant = True
        if checkLetter(guess, selectedWord, mask) == True:
            print('Correct Guess!')
        else:
            print('Incorrect guess')
        count+=1

    isVowel = False                                                   #guess single vowel
    while isVowel == False:
        print('Guess one vowel')
        guess = getGuess(guessedLetters).lower()
        if len(guess) ==1 and guess in vowels:
            isVowel = True
        else:
            print('Please enter a vowel')         
    if checkLetter(guess, selectedWord, mask) == True:
        print('Correct Guess!')
    else:
        print('Incorrect guess')
        
    print(f'roundWinner {roundWinner+1}: Guess the complete phrase to win!')        #guess final phrase
    guess = getGuess(guessedLetters).lower()
    if guess == selectedWord: 
        gameOver = True
        print('Correct guess! You Win the final round!')
        bank[rouunWinner] = bank[roundWinner] + 50000                         #if user wins pay out
        if millions[roundWinner] == True:
            bank[roundWinner] = bank[roundWinner] + 1000000                   #if user wins, and had landed on million, pay out
            
    else:
        print('Incorrect guess')
        
    print(f'Player {roundWinner+1}: ${bank[roundWinner]}')              #print final balance


In [ ]:
import random

turnList = []
bank = []
millions = []
jackpotEligible = []
roundWinner = None

#init turns, players
numPlayers = int(input('input number of players'))             #input num of players for round 1 & 2
numTurns = int(input('input number of turns'))                 #input num of turns for round 1 & 2
for x in range(numPlayers):
    turnList.append(0)
    bank.append(0)
    millions.append(False)
    jackpotEligible.append(False)             #automatically setup turn, cash, million, and jackpot tracker for each player
    

#init word, guess
wordList = []
hintList = []
loadList()                                   #loads csv word list
vowels = ['a', 'e', 'i', 'o', 'u']       

#init wheel
round = 1
wheel = loadWheel(round)                    #loads appropriate wheel for each round
    
#play 
runRound()
round+=1
runRound()
runFinalRound()
